In [1]:
import numpy as np
import matplotlib.pyplot as plt

import math
import random
import h5py
import copy

In [2]:
class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.get_virtual_cs()
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs for cross_section in self.sig_t]
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
    
energy_groups_2 = [0, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)    
        
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0. 
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  

class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    
    
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, x_size, y_size, z_size):
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
         for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
             #   print("   " + str(i))
                if boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = self.surfaces[i].distance(particle)
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):

        if particle.coordinates.x > (self.x_size/2. + self.x_0):
            particle.coordinates.x -= self.x_size/2.
        
        if particle.coordinates.y > (self.y_size/2. + self.y_0):
            particle.coordinates.y -= self.y_size/2.
        
        if particle.coordinates.z > (self.z_size/2. + self.z_0):
            particle.coordinates.z -= self.z_size/2.
            
        
        if particle.coordinates.x < (-self.x_size/2. + self.x_0):
            particle.coordinates.x += self.x_size/2.
        
        if particle.coordinates.y < (-self.y_size/2. + self.y_0):
            particle.coordinates.y += self.y_size/2.
        
        if particle.coordinates.z < (-self.z_size/2. + self.z_0):
            particle.coordinates.z += self.z_size/2.
            
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    

    

class Flux_estimator:
    def __init__(self, universe):
        self.boundaries = universe
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if self.boundaries.is_inside(particle):
            self.collision_sum[-1]  += particle.get_weight()
            
    import random

def make_initial_sources(number_of_paricles, box_size, energy=10.0e6):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources


def get_free_path(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = c_s.sig_t[energy_group_idx]
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path

def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)
    
def is_collision_virual(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    virtual_cs = c_s.virtual[energy_group_idx]
    total_cs = c_s.sig_t[energy_group_idx]
    if virtual_cs / total_cs >= random_number:
        return True
    else:
        return False
    
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)
    

def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision

def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

def process_one_particle_history(particle, universe, c_s, estimators):
    
    sum_collisions = 0.
    while not particle.is_terminated():
        free_path = get_free_path(particle, c_s)
        set_random_direction(particle)
        move_particle(particle, free_path)
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                
        for k in estimators:
            k.add_collision(particle)
            
        if is_collision_virual(particle, c_s):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, c_s)
        
            
    return particle, sum_collisions

def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])
        
        
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))
                    
                    
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles


def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])
                
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")               

In [3]:
def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std     

In [4]:
def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]/(sig_t * sum(weight_previos))
        estimator.collision_sum.append(0.)
        
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))
            

In [5]:
import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.5, 0.5, 0.5])
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
            
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, c_s, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      
    #print(estimators[1].collision_sum)
      #  assert(False)

        batch_particles = normalise_weights(batch_particles, initial_size) 
        

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators

In [6]:
cs_fission_pu_239 = [0.081600]

cs_fission_h2o = [0.0]

cs_capture_pu_239 = [0.019584]

cs_capture_h2o = [0.032640]

cs_scattering_pu_239 = [0.225216]

cs_scattering_h2o = [0.293760]

cs_total_pu_23 = [0.32640]

cs_total_h2o = [0.32640]

cs_production_neutrons_pu_239 = [2.84]

cs_production_neutrons_h2o = [0.0]

pu_23_cs_1_4 = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)

h2o_cs_1_4 = Sig(1, cs_fission_h2o, cs_capture_h2o, cs_scattering_h2o, cs_production_neutrons_h2o,
                cs_total_h2o)



energy_groups_2 = [0, 2 * 1E10]

In [7]:
estimators = []

In [8]:
class Sphere:
    def __init__(self, x_0, y_0, z_0, r):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 =z_0
        self.r = r
        
    def distance(self, particle):        
        return distance
    
    
    def get_reflected_direction(self, angle):
        
        eps = random.uniform(0, math.pi * 2)
        tetta = random.uniform(0, math.pi)
        r_reflected = Direction(eps, tetta)
        
        return r_reflected  
    
    def get_sign(self, particle):
            
        sign = ((self.x_0 - particle.coordinates.x) * (self.x_0 - particle.coordinates.x) + 
        (self.y_0 - particle.coordinates.y) * (self.y_0 - particle.coordinates.y)  + 
                (self.z_0 - particle.coordinates.z) * (self.z_0 - particle.coordinates.z) - self.r * self.r)
        
        if sign == 0:
            return 0
            
        if sign < 0:
            return -1
        else:
            return 1  
        
    def get_xml(self):

        obj_xml = " Sphere with x_0 "+ str(x_0) + " y_0 "+ str(y_0) + " z_0 " + str(z_0) + " r = "+ str(r)
        
        return obj_xml   

In [9]:
def get_sphere_estimators(radius, epsilon):
    
    estimators = []
    
    points_to_divide = [ 0.25, 0.5, 0.75]
    
    test_sphere = Sphere(0, 0, 0, 0.1)
    boundaries_type = ["black"]

    surfaces = [test_sphere]
    signs = [-1]
    
    sphere = Cell(surfaces, signs)
    estimators.append(Flux_estimator(sphere))
    
    
    for point in points_to_divide:
        
        test_sphere_inside = Sphere(0, 0, 0, epsilon + radius * point)

        test_sphere_outside = Sphere(0, 0, 0, radius * point - epsilon)

        surfaces = [test_sphere_inside, test_sphere_outside]
        signs = [-1, +1]

        sphere = Cell(surfaces, signs)
        estimators.append(Flux_estimator(sphere)) 
    
    
    test_sphere_inside = Sphere(0, 0, 0, radius)
    
    test_sphere_outside = Sphere(0, 0, 0, radius - 0.05)

    
    surfaces = [test_sphere_inside, test_sphere_outside]
    signs = [-1, +1]
    
    
    sphere = Cell(surfaces, signs)
    estimators.append(Flux_estimator(sphere))
    
    
    return estimators
    

In [10]:
test_number_of_particles = 20
test_number_interations = 100
test_number_inactive = 50

In [12]:
sphere_estimators = get_sphere_estimators(6.0825470, 0.08)
print(len(sphere_estimators))

5


In [13]:
test_sphere = Sphere(0, 0, 0, 6.082547)
boundaries_type = ["black"]

surfaces = [test_sphere]
signs = [-1]

sphere = Cell(surfaces, signs)
sphere.set_boundaries_type(boundaries_type)

In [14]:
k_effective, k_effective_exp, k_effective_std, estimators = simulation_black_boundaries(sphere,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_1_4, sphere_estimators)

 num1  9
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
 keff_cycle , k_effective_exp, std_k_effective 1.000109029989617   0.9978771010223635  0.0022319289672533826
i == 53
 keff_cycle , k_effective_exp, std_k_effective 1.0018903389795821   0.999214847008103  0.0018574354974849432
i == 54
 keff_cycle , k_effective_exp, std_k_effective 0.9796547994185062   0.9943248351107038  0.005063323974462278
i == 55
 keff_cycle , k_effective_exp, std_k_effective 1.0087351040746   0.997206888903483  0.004867091931193802
i == 56
 keff_cycle , k_effective_exp, std_k_effective 1.012721549912227   0.9997926657382736  0.004741163473642439
i == 

In [72]:
difference = (k_effective_exp[-1] - 1.) * 100000
standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  101.33296456422691  with standart deviation [pcm]+- 227.51380071126363


In [80]:
def divide_estimators_square(average_number_of_collision, radius, epsilon):
    
    average_number_of_collision[0] /= np.pi * 4/3 * 0.1 * 0.1 * 0.1
    
    average_number_of_collision[1] /= np.pi * 4/3 * ((radius * 0.25 + epsilon) * (radius * 0.25 + epsilon) * (radius * 0.25 + epsilon) -
                                (radius * 0.25 - epsilon) * (radius * 0.25 - epsilon)* (radius * 0.25 - epsilon))
    
    average_number_of_collision[2] /= np.pi * 4/3 * ((radius * 0.5 + epsilon) * (radius * 0.5 + epsilon) * (radius * 0.5 + epsilon) -
                                (radius * 0.5 - epsilon) * (radius * 0.5 - epsilon)* (radius * 0.5 - epsilon))
    
    average_number_of_collision[3] /= np.pi * 4/3 * ((radius * 0.75 + epsilon) * (radius * 0.75 + epsilon) * (radius * 0.75 + epsilon) -
                                (radius * 0.75 - epsilon) * (radius * 0.75 - epsilon)* (radius * 0.75 - epsilon))
    
    average_number_of_collision[4] /= np.pi * 4/3 * ((radius) * (radius) * (radius) -
                                            (radius - 0.05) * (radius - 0.05) * (radius - 0.05))
    
    
    
    result_estimators = [ average_number_of_collision[1]/average_number_of_collision[0], 
                         average_number_of_collision[2]/ average_number_of_collision[0],
                        average_number_of_collision[3]/ average_number_of_collision[0],
                        average_number_of_collision[4]/ average_number_of_collision[0]]
    
    return result_estimators   

In [81]:
def get_std_flux(estimators):
    
    current_std = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[20:len(current_estimator.collision_sum) - 1]
        current_std.append(get_std(array_without_zeros))
    
    return current_std  

In [82]:
def get_average_number_of_collision(estimators):
    
    average_number_of_collision = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[20:len(current_estimator.collision_sum) - 1]
        average_number_of_collision.append((sum(array_without_zeros)) / len(array_without_zeros))        
            
    return average_number_of_collision

average_sum_of_collision = get_average_number_of_collision(estimators)


In [83]:
flux_sphere = divide_estimators_square(average_sum_of_collision, 6.082547, 0.05)

In [84]:
print(flux_sphere)

[0.8124420381888587, 0.6724756166621194, 0.44063810964506755, 0.17579366590413836]


In [85]:
flux_std = get_std_flux(estimators)

print(flux_std)

test_flux = [0.93538006, 0.75575352, 0.49884364, 0.19222603]

for i in range(0, len(test_flux)):
    print(str(((flux_sphere[i] - test_flux[i])/test_flux[i] )* 100) + " % " + str(flux_std[i + 1] * 100000))

[1.4235552452273967e-05, 0.0003352335794748197, 0.0007112692421113317, 0.0007972722397175421, 0.0004999436677431664]
-13.143109102747093 % 33.52335794748197
-11.019188284810179 % 71.12692421113317
-11.668091098632113 % 79.7272239717542
-8.548459381833787 % 49.99436677431664


In [86]:
def save_flux_to_file(estimators, file_name):
        
    k = 0
    for estimator in estimators:
        
        result_file_name = file_name + str(k)+".txt"
        
        f = open(result_file_name, "w")
        array_without_zeros = estimator.collision_sum
        for i in range(0, len(array_without_zeros)):
            f.write(" flux "+ str(array_without_zeros[i]) +'\n')
        k += 1
        
        f.close()


In [87]:
file_name = "result_flux.txt"
save_flux_to_file(estimators, file_name)

In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import matplotlib.pyplot as plt

import math
import random
import h5py
import copy


# In[4]:


class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.get_virtual_cs()
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs for cross_section in self.sig_t]
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
    
energy_groups_2 = [0, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)    
        
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0. 
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  

class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    
    
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, x_size, y_size, z_size):
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
         for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
             #   print("   " + str(i))
                if boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = self.surfaces[i].distance(particle)
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):

        if particle.coordinates.x > (self.x_size/2. + self.x_0):
            particle.coordinates.x -= self.x_size/2.
        
        if particle.coordinates.y > (self.y_size/2. + self.y_0):
            particle.coordinates.y -= self.y_size/2.
        
        if particle.coordinates.z > (self.z_size/2. + self.z_0):
            particle.coordinates.z -= self.z_size/2.
            
        
        if particle.coordinates.x < (-self.x_size/2. + self.x_0):
            particle.coordinates.x += self.x_size/2.
        
        if particle.coordinates.y < (-self.y_size/2. + self.y_0):
            particle.coordinates.y += self.y_size/2.
        
        if particle.coordinates.z < (-self.z_size/2. + self.z_0):
            particle.coordinates.z += self.z_size/2.
            
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    

    

class Flux_estimator:
    def __init__(self, universe):
        self.boundaries = universe
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if self.boundaries.is_inside(particle):
            self.collision_sum[-1]  += particle.get_weight()
            
    import random

def make_initial_sources(number_of_paricles, box_size, energy=10.0e6):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources


def get_free_path(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = c_s.sig_t[energy_group_idx]
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path

def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)
    
def is_collision_virual(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    virtual_cs = c_s.virtual[energy_group_idx]
    total_cs = c_s.sig_t[energy_group_idx]
    if virtual_cs / total_cs >= random_number:
        return True
    else:
        return False
    
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)
    

def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision

def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

def process_one_particle_history(particle, universe, c_s, estimators):
    
    sum_collisions = 0.
    while not particle.is_terminated():
        free_path = get_free_path(particle, c_s)
        set_random_direction(particle)
        move_particle(particle, free_path)
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                
        for k in estimators:
            k.add_collision(particle)
            
        if is_collision_virual(particle, c_s):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, c_s)
        
            
    return particle, sum_collisions

def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        
        
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))
                    
                    
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles


def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])
                
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")               


# In[5]:


def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std     


# In[6]:


def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]/(sig_t * sum(weight_previos))
        estimator.collision_sum.append(0.)
        
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))
            


# In[7]:


import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.5, 0.5, 0.5])
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
            
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, c_s, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      
    #print(estimators[1].collision_sum)
      #  assert(False)

        batch_particles = normalise_weights(batch_particles, initial_size) 
        

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators


# In[8]:


cs_fission_pu_239 = [0.081600]

cs_fission_h2o = [0.0]

cs_capture_pu_239 = [0.019584]

cs_capture_h2o = [0.032640]

cs_scattering_pu_239 = [0.225216]

cs_scattering_h2o = [0.293760]

cs_total_pu_23 = [0.32640]

cs_total_h2o = [0.32640]

cs_production_neutrons_pu_239 = [2.84]

cs_production_neutrons_h2o = [0.0]

pu_23_cs_1_4 = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)

h2o_cs_1_4 = Sig(1, cs_fission_h2o, cs_capture_h2o, cs_scattering_h2o, cs_production_neutrons_h2o,
                cs_total_h2o)



energy_groups_2 = [0, 2 * 1E10]


# In[9]:


estimators = []


# In[10]:


class Sphere:
    def __init__(self, x_0, y_0, z_0, r):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 =z_0
        self.r = r
        
    def distance(self, particle):        
        return distance
    
    
    def get_reflected_direction(self, angle):
        
        eps = random.uniform(0, math.pi * 2)
        tetta = random.uniform(0, math.pi)
        r_reflected = Direction(eps, tetta)
        
        return r_reflected  
    
    def get_sign(self, particle):
            
        sign = ((self.x_0 - particle.coordinates.x) * (self.x_0 - particle.coordinates.x) + 
        (self.y_0 - particle.coordinates.y) * (self.y_0 - particle.coordinates.y)  + 
                (self.z_0 - particle.coordinates.z) * (self.z_0 - particle.coordinates.z) - self.r * self.r)
        
        if sign == 0:
            return 0
            
        if sign < 0:
            return -1
        else:
            return 1  
        
    def get_xml(self):

        obj_xml = " Sphere with x_0 "+ str(x_0) + " y_0 "+ str(y_0) + " z_0 " + str(z_0) + " r = "+ str(r)
        
        return obj_xml   


# In[54]:


def get_sphere_estimators(radius, epsilon):
    
    estimators = []
    
    points_to_divide = [ 0.25, 0.5, 0.75]
    
    test_sphere = Sphere(0, 0, 0, epsilon)
    boundaries_type = ["black"]

    surfaces = [test_sphere]
    signs = [-1]
    
    sphere = Cell(surfaces, signs)
    estimators.append(Flux_estimator(sphere))
    
    
    for point in points_to_divide:
        
        test_sphere_inside = Sphere(0, 0, 0, epsilon + radius * point)

        test_sphere_outside = Sphere(0, 0, 0, radius * point - epsilon)

        surfaces = [test_sphere_inside, test_sphere_outside]
        signs = [-1, +1]

        sphere = Cell(surfaces, signs)
        estimators.append(Flux_estimator(sphere)) 
    
    
    test_sphere_inside = Sphere(0, 0, 0, radius)
    
    test_sphere_outside = Sphere(0, 0, 0, radius - 0.05)

    
    surfaces = [test_sphere_inside, test_sphere_outside]
    signs = [-1, +1]
    
    
    sphere = Cell(surfaces, signs)
    estimators.append(Flux_estimator(sphere))
    
    
    return estimators
    


# In[55]:


test_number_of_particles = 10
test_number_interations = 500
test_number_inactive = 50


# In[56]:


sphere_estimators = get_sphere_estimators(6.0825470, 0.1)
print(len(sphere_estimators))


# In[57]:


test_sphere = Sphere(0, 0, 0, 6.082547)
boundaries_type = ["black"]

surfaces = [test_sphere]
signs = [-1]

sphere = Cell(surfaces, signs)
sphere.set_boundaries_type(boundaries_type)


# In[58]:


k_effective, k_effective_exp, k_effective_std, estimators = simulation_black_boundaries(sphere,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_1_4, sphere_estimators)


# In[59]:


def divide_estimators_square(average_number_of_collision, radius, epsilon):
    
    average_number_of_collision[0] /= np.pi * 4/3 * epsilon * epsilon * epsilon
    
    average_number_of_collision[1] /= np.pi * 4/3 * ((radius * 0.25 + epsilon) * (radius * 0.25 + epsilon) * (radius * 0.25 + epsilon) -
                                (radius * 0.25 - epsilon) * (radius * 0.25 - epsilon)* (radius * 0.25 - epsilon))
    
    average_number_of_collision[2] /= np.pi * 4/3 * ((radius * 0.5 + epsilon) * (radius * 0.5 + epsilon) * (radius * 0.5 + epsilon) -
                                (radius * 0.5 - epsilon) * (radius * 0.5 - epsilon)* (radius * 0.5 - epsilon))
    
    average_number_of_collision[3] /= np.pi * 4/3 * ((radius * 0.75 + epsilon) * (radius * 0.75 + epsilon) * (radius * 0.75 + epsilon) -
                                (radius * 0.75 - epsilon) * (radius * 0.75 - epsilon)* (radius * 0.75 - epsilon))
    
    average_number_of_collision[4] /= np.pi * 4/3 * ((radius) * (radius) * (radius) -
                                            (radius - 0.05) * (radius - 0.05) * (radius - 0.05))
    
    
    
    result_estimators = [ average_number_of_collision[1]/average_number_of_collision[0], 
                         average_number_of_collision[2]/ average_number_of_collision[0],
                        average_number_of_collision[3]/ average_number_of_collision[0],
                        average_number_of_collision[4]/ average_number_of_collision[0]]
    
    return result_estimators   


# In[60]:


def get_std_flux(estimators):
    
    current_std = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[20:len(current_estimator.collision_sum) - 1]
        current_std.append(get_std(array_without_zeros))
    
    return current_std  


# In[61]:


def get_average_number_of_collision(estimators):
    
    average_number_of_collision = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[20:len(current_estimator.collision_sum) - 1]
        average_number_of_collision.append((sum(array_without_zeros)) / len(array_without_zeros))        
            
    return average_number_of_collision

average_sum_of_collision = get_average_number_of_collision(estimators)


# In[62]:


flux_sphere = divide_estimators_square(average_sum_of_collision, 6.082547, 0.1)


# In[63]:


print(flux_sphere)


# In[64]:


flux_std = get_std_flux(estimators)

print(flux_std)

test_flux = [0.93538006, 0.75575352, 0.49884364, 0.19222603]

for i in range(0, len(test_flux)):
    print(str(((flux_sphere[i] - test_flux[i])/test_flux[i] )* 100) + " % " + str(flux_std[i + 1] * 100000))


# In[65]:


def save_flux_to_file(estimators, file_name):
        
    k = 0
    for estimator in estimators:
        
        result_file_name = file_name + str(k)+".txt"
        
        f = open(result_file_name, "w")
        array_without_zeros = estimator.collision_sum
        for i in range(0, len(array_without_zeros)):
            f.write(" flux "+ str(array_without_zeros[i]) +'\n')
        k += 1
        
        f.close()


# In[66]:


file_name = "result_flux"
save_flux_to_file(estimators, file_name)


def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()


# In[58]:


file_name = "output"
save_simulation_results_to_file(k_effective, file_name)





5
 num1  6
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
 keff_cycle , k_effective_exp, std_k_effective 1.0106129151578693   0.9982743647300414  0.012338550427827888
i == 53
 keff_cycle , k_effective_exp, std_k_effective 0.9437379628260312   0.9800955640953714  0.019524738190351377
i == 54
 keff_cycle , k_effective_exp, std_k_effective 0.9683335551850556   0.9771550618677924  0.014115744898916227
i == 55
 keff_cycle , k_effective_exp, std_k_effective 1.0668530835491816   0.9950946662040703  0.021009092226294938
i == 56
 keff_cycle , k_effective_exp, std_k_effective 1.0862993725802714   1.0102954506001038  0.02291982730526869

 keff_cycle , k_effective_exp, std_k_effective 0.9728438482438365   1.0002044152461185  0.0052180631582053015
i == 120
 keff_cycle , k_effective_exp, std_k_effective 0.9959967788232496   1.0001443061543633  0.005143330451828076
i == 121
 keff_cycle , k_effective_exp, std_k_effective 0.9980823345184957   1.000115264300337  0.005070454885435341
i == 122
 keff_cycle , k_effective_exp, std_k_effective 0.9346350162756981   0.999205816411106  0.00508157997837778
i == 123
 keff_cycle , k_effective_exp, std_k_effective 1.0263717795615435   0.9995779528926187  0.0050252836850146814
i == 124
 keff_cycle , k_effective_exp, std_k_effective 0.9663094492762869   0.999128378519425  0.004977254951318459
i == 125
 keff_cycle , k_effective_exp, std_k_effective 1.0499099021453715   0.9998054655011043  0.004956904126082365
i == 126
 keff_cycle , k_effective_exp, std_k_effective 1.0251171024630568   1.000138513355867  0.00490257246720839
i == 127
 keff_cycle , k_effective_exp, std_k_effective 1.06743587970

 keff_cycle , k_effective_exp, std_k_effective 0.9750686740425792   0.9976275592567657  0.0034941229052680464
i == 190
 keff_cycle , k_effective_exp, std_k_effective 1.012643385169257   0.9977348151561407  0.0034707327642892216
i == 191
 keff_cycle , k_effective_exp, std_k_effective 1.0196124826373993   0.9978899759184191  0.003449521087735967
i == 192
 keff_cycle , k_effective_exp, std_k_effective 1.0033103975192355   0.9979281479015235  0.0034253552412715125
i == 193
 keff_cycle , k_effective_exp, std_k_effective 1.0055248092615483   0.9979812714075377  0.0034017321933264314
i == 194
 keff_cycle , k_effective_exp, std_k_effective 0.9094359413987947   0.9973663732824769  0.003433534975218859
i == 195
 keff_cycle , k_effective_exp, std_k_effective 0.9363711171526586   0.9969457163436505  0.003435623022003849
i == 196
 keff_cycle , k_effective_exp, std_k_effective 0.9674883800940541   0.9967439537665985  0.0034179704255568825
i == 197
 keff_cycle , k_effective_exp, std_k_effective 1.000

 keff_cycle , k_effective_exp, std_k_effective 0.9938047794753212   0.9997238785386339  0.0029182761258494394
i == 260
 keff_cycle , k_effective_exp, std_k_effective 0.8712965248202068   0.9991123197114032  0.002968035004557534
i == 261
 keff_cycle , k_effective_exp, std_k_effective 0.994553391296992   0.9990907134156003  0.002954014013663462
i == 262
 keff_cycle , k_effective_exp, std_k_effective 0.9191050021274244   0.9987134223246183  0.0029641566642356447
i == 263
 keff_cycle , k_effective_exp, std_k_effective 1.004110970024747   0.9987387629241495  0.0029503164429248827
i == 264
 keff_cycle , k_effective_exp, std_k_effective 0.934542125688396   0.9984387786380012  0.0029517805575567566
i == 265
 keff_cycle , k_effective_exp, std_k_effective 1.0907549407483106   0.9988681561361886  0.002969229235749864
i == 266
 keff_cycle , k_effective_exp, std_k_effective 0.9964568959464226   0.9988569928945693  0.0029554719181423827
i == 267
 keff_cycle , k_effective_exp, std_k_effective 1.02937

 keff_cycle , k_effective_exp, std_k_effective 0.9464382506425486   1.0002257069072498  0.0025690173872974908
i == 330
 keff_cycle , k_effective_exp, std_k_effective 1.0104693034562755   1.0002622911806391  0.0025600872948359156
i == 331
 keff_cycle , k_effective_exp, std_k_effective 0.9871644249601929   1.000215679557079  0.0025513862074534417
i == 332
 keff_cycle , k_effective_exp, std_k_effective 1.0191272404581038   1.0002827418297775  0.0025432069820759532
i == 333
 keff_cycle , k_effective_exp, std_k_effective 1.0197808662296253   1.0003516397958547  0.002535140850205352
i == 334
 keff_cycle , k_effective_exp, std_k_effective 1.000246818425969   1.000351270706524  0.0025261985535357
i == 335
 keff_cycle , k_effective_exp, std_k_effective 0.9798695780315462   1.000279405118191  0.002518344710611741
i == 336
 keff_cycle , k_effective_exp, std_k_effective 1.0274253202854888   1.0003743209054892  0.0025113181832744362
i == 337
 keff_cycle , k_effective_exp, std_k_effective 0.92163014

 keff_cycle , k_effective_exp, std_k_effective 0.9750232202373565   1.0006579008187175  0.0022750460330107996
i == 400
 keff_cycle , k_effective_exp, std_k_effective 1.087762177213847   1.0009067701798464  0.002282146843293944
i == 401
 keff_cycle , k_effective_exp, std_k_effective 1.011361884374583   1.000936556829974  0.002275830648043172
i == 402
 keff_cycle , k_effective_exp, std_k_effective 0.9974060560954577   1.0009265269983418  0.0022693781742600394
i == 403
 keff_cycle , k_effective_exp, std_k_effective 0.963878531767951   1.0008215751704936  0.002265372657152376
i == 404
 keff_cycle , k_effective_exp, std_k_effective 0.9373789009210667   1.0006423585765687  0.002266062223990455
i == 405
 keff_cycle , k_effective_exp, std_k_effective 1.0024311328127626   1.000647397377234  0.0022596755521407347
i == 406
 keff_cycle , k_effective_exp, std_k_effective 0.9547528027077007   1.0005184799764768  0.002257004021559329
i == 407
 keff_cycle , k_effective_exp, std_k_effective 0.975683335

 keff_cycle , k_effective_exp, std_k_effective 0.9948381659598623   0.9996629891938279  0.0020952485631408873
i == 470
 keff_cycle , k_effective_exp, std_k_effective 0.9515886951917123   0.999548526589061  0.0020933855713808867
i == 471
 keff_cycle , k_effective_exp, std_k_effective 1.0057694592375956   0.9995633031511715  0.002088459514808641
i == 472
 keff_cycle , k_effective_exp, std_k_effective 1.0211290778595503   0.999614406882708  0.0020841313171340524
i == 473
 keff_cycle , k_effective_exp, std_k_effective 0.9994691870932291   0.9996140635735129  0.002079198483371788
i == 474
 keff_cycle , k_effective_exp, std_k_effective 0.9913058716212471   0.9995944687811726  0.002074381466079883
i == 475
 keff_cycle , k_effective_exp, std_k_effective 1.0199999886530156   0.9996424817691064  0.002070051696678947
i == 476
 keff_cycle , k_effective_exp, std_k_effective 0.98254145869715   0.9996023385224586  0.002065576820393065
i == 477
 keff_cycle , k_effective_exp, std_k_effective 0.97761138

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import matplotlib.pyplot as plt

import math
import random
import h5py
import copy


# In[2]:


class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.get_virtual_cs()
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs for cross_section in self.sig_t]
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
    
energy_groups_2 = [0, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)    
        
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0. 
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  

class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    
    
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, x_size, y_size, z_size):
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
         for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
             #   print("   " + str(i))
                if boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = self.surfaces[i].distance(particle)
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):

        if particle.coordinates.x > (self.x_size/2. + self.x_0):
            particle.coordinates.x -= self.x_size/2.
        
        if particle.coordinates.y > (self.y_size/2. + self.y_0):
            particle.coordinates.y -= self.y_size/2.
        
        if particle.coordinates.z > (self.z_size/2. + self.z_0):
            particle.coordinates.z -= self.z_size/2.
            
        
        if particle.coordinates.x < (-self.x_size/2. + self.x_0):
            particle.coordinates.x += self.x_size/2.
        
        if particle.coordinates.y < (-self.y_size/2. + self.y_0):
            particle.coordinates.y += self.y_size/2.
        
        if particle.coordinates.z < (-self.z_size/2. + self.z_0):
            particle.coordinates.z += self.z_size/2.
            
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    

    

class Flux_estimator:
    def __init__(self, universe):
        self.boundaries = universe
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if self.boundaries.is_inside(particle):
            self.collision_sum[-1]  += particle.get_weight()
            
    import random

def make_initial_sources(number_of_paricles, box_size, energy=10.0e6):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources


def get_free_path(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = c_s.sig_t[energy_group_idx]
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path

def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)
    
def is_collision_virual(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    virtual_cs = c_s.virtual[energy_group_idx]
    total_cs = c_s.sig_t[energy_group_idx]
    if virtual_cs / total_cs >= random_number:
        return True
    else:
        return False
    
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)
    

def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision

def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

def process_one_particle_history(particle, universe, c_s, estimators):
    
    sum_collisions = 0.
    while not particle.is_terminated():
        free_path = get_free_path(particle, c_s)
        set_random_direction(particle)
        move_particle(particle, free_path)
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                
        for k in estimators:
            k.add_collision(particle)
            
        if is_collision_virual(particle, c_s):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, c_s)
        
            
    return particle, sum_collisions

def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])
        
        
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))
                    
                    
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles


def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])
                
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")               


# In[3]:


def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std     


# In[4]:


def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]/(sig_t * sum(weight_previos))
        estimator.collision_sum.append(0.)
        
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))
            


# In[5]:


import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.5, 0.5, 0.5])
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
            
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, c_s, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      
    #print(estimators[1].collision_sum)
      #  assert(False)

        batch_particles = normalise_weights(batch_particles, initial_size) 
        

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators


# In[6]:


cs_fission_pu_239 = [0.081600]

cs_fission_h2o = [0.0]

cs_capture_pu_239 = [0.019584]

cs_capture_h2o = [0.032640]

cs_scattering_pu_239 = [0.225216]

cs_scattering_h2o = [0.293760]

cs_total_pu_23 = [0.32640]

cs_total_h2o = [0.32640]

cs_production_neutrons_pu_239 = [2.84]

cs_production_neutrons_h2o = [0.0]

pu_23_cs_1_4 = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)

h2o_cs_1_4 = Sig(1, cs_fission_h2o, cs_capture_h2o, cs_scattering_h2o, cs_production_neutrons_h2o,
                cs_total_h2o)



energy_groups_2 = [0, 2 * 1E10]


# In[7]:


estimators = []


# In[66]:


class Sphere:
    def __init__(self, x_0, y_0, z_0, r):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 =z_0
        self.r = r
        
    def distance(self, particle):        
        return distance
    
    
    def get_reflected_direction(self, angle):
        
        eps = random.uniform(0, math.pi * 2)
        tetta = random.uniform(0, math.pi)
        r_reflected = Direction(eps, tetta)
        
        return r_reflected  
    
    def get_sign(self, particle):
            
        sign = ((self.x_0 - particle.coordinates.x) * (self.x_0 - particle.coordinates.x) + 
        (self.y_0 - particle.coordinates.y) * (self.y_0 - particle.coordinates.y)  + 
                (self.z_0 - particle.coordinates.z) * (self.z_0 - particle.coordinates.z) - self.r * self.r)
        
        if sign == 0:
            return 0
            
        if sign < 0:
            return -1
        else:
            return 1  
        
    def get_xml(self):

        obj_xml = " Sphere with x_0 "+ str(x_0) + " y_0 "+ str(y_0) + " z_0 " + str(z_0) + " r = "+ str(r)
        
        return obj_xml   


# In[67]:


def get_sphere_estimators(radius, epsilon):
    
    estimators = []
    
    points_to_divide = [ 0.25, 0.5, 0.75]
    
    test_sphere = Sphere(0, 0, 0, 0.1)
    boundaries_type = ["black"]

    surfaces = [test_sphere]
    signs = [-1]
    
    sphere = Cell(surfaces, signs)
    estimators.append(Flux_estimator(sphere))
    
    
    for point in points_to_divide:
        
        test_sphere_inside = Sphere(0, 0, 0, epsilon + radius * point)

        test_sphere_outside = Sphere(0, 0, 0, radius * point - epsilon)

        surfaces = [test_sphere_inside, test_sphere_outside]
        signs = [-1, +1]

        sphere = Cell(surfaces, signs)
        estimators.append(Flux_estimator(sphere)) 
    
    
    test_sphere_inside = Sphere(0, 0, 0, radius)
    
    test_sphere_outside = Sphere(0, 0, 0, radius - 0.05)

    
    surfaces = [test_sphere_inside, test_sphere_outside]
    signs = [-1, +1]
    
    
    sphere = Cell(surfaces, signs)
    estimators.append(Flux_estimator(sphere))
    
    
    return estimators
    


# In[68]:


test_number_of_particles = 60
test_number_interations = 600
test_number_inactive = 100


# In[69]:


sphere_estimators = get_sphere_estimators(6.0825470, 0.05)
print(len(sphere_estimators))


# In[70]:


test_sphere = Sphere(0, 0, 0, 6.082547)
boundaries_type = ["black"]

surfaces = [test_sphere]
signs = [-1]

sphere = Cell(surfaces, signs)
sphere.set_boundaries_type(boundaries_type)


# In[71]:


k_effective, k_effective_exp, k_effective_std, estimators = simulation_black_boundaries(sphere,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_1_4, sphere_estimators)


# In[72]:


difference = (k_effective_exp[-1] - 1.) * 100000
standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


# In[80]:


def divide_estimators_square(average_number_of_collision, radius, epsilon):
    
    average_number_of_collision[0] /= np.pi * 4/3 * 0.1 * 0.1 * 0.1
    
    average_number_of_collision[1] /= np.pi * 4/3 * ((radius * 0.25 + epsilon) * (radius * 0.25 + epsilon) * (radius * 0.25 + epsilon) -
                                (radius * 0.25 - epsilon) * (radius * 0.25 - epsilon)* (radius * 0.25 - epsilon))
    
    average_number_of_collision[2] /= np.pi * 4/3 * ((radius * 0.5 + epsilon) * (radius * 0.5 + epsilon) * (radius * 0.5 + epsilon) -
                                (radius * 0.5 - epsilon) * (radius * 0.5 - epsilon)* (radius * 0.5 - epsilon))
    
    average_number_of_collision[3] /= np.pi * 4/3 * ((radius * 0.75 + epsilon) * (radius * 0.75 + epsilon) * (radius * 0.75 + epsilon) -
                                (radius * 0.75 - epsilon) * (radius * 0.75 - epsilon)* (radius * 0.75 - epsilon))
    
    average_number_of_collision[4] /= np.pi * 4/3 * ((radius) * (radius) * (radius) -
                                            (radius - 0.05) * (radius - 0.05) * (radius - 0.05))
    
    
    
    result_estimators = [ average_number_of_collision[1]/average_number_of_collision[0], 
                         average_number_of_collision[2]/ average_number_of_collision[0],
                        average_number_of_collision[3]/ average_number_of_collision[0],
                        average_number_of_collision[4]/ average_number_of_collision[0]]
    
    return result_estimators   


# In[81]:


def get_std_flux(estimators):
    
    current_std = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[20:len(current_estimator.collision_sum) - 1]
        current_std.append(get_std(array_without_zeros))
    
    return current_std  


# In[82]:


def get_average_number_of_collision(estimators):
    
    average_number_of_collision = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[20:len(current_estimator.collision_sum) - 1]
        average_number_of_collision.append((sum(array_without_zeros)) / len(array_without_zeros))        
            
    return average_number_of_collision

average_sum_of_collision = get_average_number_of_collision(estimators)


# In[83]:


flux_sphere = divide_estimators_square(average_sum_of_collision, 6.082547, 0.05)


# In[84]:


print(flux_sphere)


# In[85]:


flux_std = get_std_flux(estimators)

print(flux_std)

test_flux = [0.93538006, 0.75575352, 0.49884364, 0.19222603]

for i in range(0, len(test_flux)):
    print(str(((flux_sphere[i] - test_flux[i])/test_flux[i] )* 100) + " % " + str(flux_std[i + 1] * 100000))


# In[86]:


def save_flux_to_file(estimators, file_name):
        
    k = 0
    for estimator in estimators:
        
        result_file_name = file_name + str(k)+".txt"
        
        f = open(result_file_name, "w")
        array_without_zeros = estimator.collision_sum
        for i in range(0, len(array_without_zeros)):
            f.write(" flux "+ str(array_without_zeros[i]) +'\n')
        k += 1
        
        f.close()


def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()


# In[58]:


file_name = "output"
save_simulation_results_to_file(k_effective, file_name)


file_name = "result_flux.txt"
save_flux_to_file(estimators, file_name)